In [1]:
import jax
import jax.numpy as jnp

In [2]:
from jax_smi import initialise_tracking
initialise_tracking()
# some computation...

In [3]:
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

# Issue: Eigen-decomposition with TPU fails

In [29]:
import jax
import jax.numpy as jnp

def eig(X, type_complex):
    import numpy as np
    jax.config.update('jax_enable_x64', True)

    type_complex = jnp.complex64
    X = jnp.array(X, dtype=jnp.complex64)
    _eig = lambda x: jax.jit(np.linalg.eig, device=jax.devices('cpu')[0])(x)

    eigenvalues_shape = jax.ShapeDtypeStruct(X.shape[:-1], type_complex)
    eigenvectors_shape = jax.ShapeDtypeStruct(X.shape, type_complex)

    result_shape_dtype = (eigenvalues_shape, eigenvectors_shape)
    return jax.pure_callback(_eig, result_shape_dtype, X)

X = jnp.arange(1000*1000).reshape((1000,1000))
eig(X, jnp.complex64)


NotImplementedError: 64-bit types not supported.

# Example: Convolve

In [5]:
import numpy as np
import jax.numpy as jnp

size1 = 1000
size2 = 2+101

x = np.arange(size1)
w = np.arange(2, size2, 1)
# w = np.array([2., 3., 4.])

def convolve(x, w):
    output = []
    for i in range(size2//2, len(x)-size2//2):
        output.append(jnp.dot(x[i-size2//2:i+size2//2-1], w))
    return jnp.array(output)

# convolve(x, w)

In [7]:
n_devices = jax.local_device_count()
xs = np.arange(size1 * n_devices).reshape(-1, size1)
ws = np.stack([w] * n_devices)

array([[   0,    1,    2, ...,  997,  998,  999],
       [1000, 1001, 1002, ..., 1997, 1998, 1999],
       [2000, 2001, 2002, ..., 2997, 2998, 2999],
       ...,
       [5000, 5001, 5002, ..., 5997, 5998, 5999],
       [6000, 6001, 6002, ..., 6997, 6998, 6999],
       [7000, 7001, 7002, ..., 7997, 7998, 7999]])

In [15]:
jax.vmap(jax.jit(convolve))(xs, ws)

Array([[  348450,   353702,   358954, ...,  5048990,  5054242,  5059494],
       [ 5600450,  5605702,  5610954, ..., 10300990, 10306242, 10311494],
       [10852450, 10857702, 10862954, ..., 15552990, 15558242, 15563494],
       ...,
       [26608450, 26613702, 26618954, ..., 31308990, 31314242, 31319494],
       [31860450, 31865702, 31870954, ..., 36560990, 36566242, 36571494],
       [37112450, 37117702, 37122954, ..., 41812990, 41818242, 41823494]],      dtype=int32)

In [2]:
jax.vmap(convolve)(xs+3, ws)

NameError: name 'jax' is not defined

In [1]:
%timeit jax.pmap(convolve)(xs+10, ws+3)

NameError: name 'jax' is not defined